# Notebook for Generating dataframes from multiple sessions

Pilot 3 added more checks for engagement early in the experiment.
Iteration names:
'pre-pilot3':

## 1. Read in packages and set up server connection
This first section will read in necessary packages for anaysis and establish file paths and connections to the mongo db server

In [1]:
import os
import sys
import urllib, io
os.getcwd()
sys.path.append("..")
sys.path.append("../utils")
sys.path.append("../analysis/utils")


import numpy as np
import scipy.stats as stats
import pandas as pd

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import drawing_utils as drawing
import importlib
import scoring

In [2]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir =  os.path.abspath('.')
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'behavioral_experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

### establish connection to mongo
First thing you need to do is to establish an ssh tunnel (aka remote port forwarding) to the server, so that requests to the mongodb can be made "as if" the mongodb server is running on your local computer. Run this from the command line before you begin data analysis if you plan to fetch data from mongo:

`ssh -fNL 27017:127.0.0.1:27017 USERNAME@cogtoolslab.org`

In [14]:
# set vars 
auth = pd.read_csv(os.path.join(analysis_dir,'../auth.txt'), header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org'

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['compositional-abstractions']
coll = db['prior-elicitation']

# which iteration name should we use?
iterationName = 'testPilot'

# ## look up number of trials (including paired-practice)
# numTrials = 13

In [15]:
df_all = pd.DataFrame(coll.find())
print(df_all.columns)

Index(['_id', 'rt', 'url', 'trial_type', 'trial_index', 'time_elapsed',
       'internal_node_id', 'wID', 'aID', 'hitID', 'iterationName', 'stimulus',
       'button_pressed', 'test_part', 'utterance', 'target', 'responses',
       'competitor_type', 'target_type', 'pptID', 'question_order', 'workerId',
       'gameID', 'eventType'],
      dtype='object')


In [16]:
df_responses = df_all[(df_all['trial_type'] == 'survey-text') & (df_all['iterationName'] == iterationName)]
print(df_responses.shape)
df_responses.head()

(2, 24)


,_id,rt,url,trial_type,trial_index,time_elapsed,internal_node_id,wID,aID,hitID,...,utterance,target,responses,competitor_type,target_type,pptID,question_order,workerId,gameID,eventType
235,609086b709bb643db4f5a15a,3021.385,NaN,survey-text,2,8656,0.0-2.0-0.0,,,,...,block,tower_0.png,"{""Q0"":""block""}",NaN,NaN,0342-2ca2d145-4e56-61ed-9a56-15f51d3b527a,NaN,NaN,NaN,NaN
236,609086ba09bb643db4f5a15b,2454.250,NaN,survey-text,3,11618,0.0-2.0-0.1,,,,...,house,tower_1.png,"{""Q0"":""house""}",NaN,NaN,7678-0046c019-f8ec-658c-9013-31951db1279e,NaN,NaN,NaN,NaN
